In [2]:
import sys
import json
import pickle
import pyspark
from functools import reduce
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf, mean, lit, stddev, col, expr, when
from pyspark.sql.types import DoubleType, ArrayType, ShortType, LongType, IntegerType
import pandas as pd
from collections import OrderedDict
from datetime import date
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

from pyspark.ml import Pipeline, PipelineModel

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml import PipelineModel

In [3]:
#sc = SparkContext.getOrCreate()
sql = sqlContext #SQLContext.getOrCreate(sc)

In [4]:
STORAGE_ACCOUNT_SUFFIX = 'core.windows.net'
STORAGE_ACCOUNT_NAME = os.environ['STORAGE_ACCOUNT_NAME']

wasbUrlOutput = "wasb://{0}@{1}.blob.{2}/features.parquet".format(
            'intermediate',
            STORAGE_ACCOUNT_NAME,
            STORAGE_ACCOUNT_SUFFIX)

dfa = sql.read.parquet(wasbUrlOutput)
dfa.printSchema()

root
 |-- EnqueuedTimeUtc: string (nullable = true)
 |-- machineID: string (nullable = true)
 |-- ambient_temperature: double (nullable = true)
 |-- ambient_pressure: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- temperature: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- f0: double (nullable = true)
 |-- f1: double (nullable = true)
 |-- f2: double (nullable = true)
 |-- a0: double (nullable = true)
 |-- a1: double (nullable = true)
 |-- a2: double (nullable = true)
 |-- temperature_n: double (nullable = true)
 |-- pressure_n: double (nullable = true)
 |-- f0_n: double (nullable = true)
 |-- f1_n: double (nullable = true)
 |-- f2_n: double (nullable = true)
 |-- a0_n: double (nullable = true)
 |-- a1_n: double (nullable = true)
 |-- a2_n: double (nullable = true)



In [5]:
features = sorted([c for c in dfa.columns if c not in ['machineID', 'EnqueuedTimeUtc']])

# assemble features
va = VectorAssembler(inputCols=features, outputCol='features')

# this is a hack!! In the current simulated dataset, there are 4 machines: machine 0 is "good," 
# whereas the rest are experiencing different failures. This will eventually be produced by merging 
# telemetry with the "maintenance log"
feat_data = va.transform(dfa).withColumn('label_e', dfa.machineID.substr(-1, 1).cast(IntegerType()))
dfa.unpersist()
feat_data.persist(pyspark.StorageLevel.DISK_ONLY)
feat_data.limit(10).toPandas().head()

,EnqueuedTimeUtc,machineID,ambient_temperature,ambient_pressure,speed,temperature,pressure,f0,f1,f2,...,temperature_n,pressure_n,f0_n,f1_n,f2_n,a0_n,a1_n,a2_n,features,label_e
0,2018-03-19T02:49:40.9200000Z,pm1-353,20.0,101.0,1000.0,93.984369,1999.998863,200.0,67.0,83.0,...,0.093984,1.999999,0.2,0.06700,0.08300,2.130736,1.234923,1.104255,"[2130.73611212, 2.13073611212, 1234.92292531, ...",3
1,2018-03-19T02:49:40.9820000Z,pm1-356,20.0,101.0,800.0,74.463702,1199.999534,160.0,53.0,27.0,...,0.093080,1.499999,0.2,0.06625,0.03375,2.660304,1.501537,1.342148,"[2128.24325483, 2.66030406854, 1201.22920143, ...",6
2,2018-03-19T02:49:41.9410000Z,pm1-353,20.0,101.0,1000.0,93.985845,1999.998864,200.0,67.0,33.0,...,0.093986,1.999999,0.2,0.06700,0.03300,2.128995,1.209310,1.103185,"[2128.99464918, 2.12899464918, 1209.30977722, ...",3
3,2018-03-19T02:49:41.9880000Z,pm1-356,20.0,101.0,800.0,74.465005,1199.999534,160.0,53.0,67.0,...,0.093081,1.499999,0.2,0.06625,0.08375,2.663272,1.539738,1.375497,"[2130.61728994, 2.66327161242, 1231.79044275, ...",6
4,2018-03-19T02:49:42.9280000Z,pm1-356,20.0,101.0,800.0,74.466307,1199.999535,160.0,53.0,67.0,...,0.093083,1.499999,0.2,0.06625,0.08375,2.663159,1.539378,1.375902,"[2130.52747399, 2.66315934249, 1231.50251145, ...",6


In [6]:
# set maxCategories so features with > 10 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(feat_data)

# fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol="label_e", outputCol="indexedLabel").fit(feat_data)

In [7]:
training, test = feat_data.randomSplit([0.8, 0.2], seed=12345)
print(training.count())
print(test.count())
feat_data.limit(5).toPandas().head()

56377
14171


,EnqueuedTimeUtc,machineID,ambient_temperature,ambient_pressure,speed,temperature,pressure,f0,f1,f2,...,temperature_n,pressure_n,f0_n,f1_n,f2_n,a0_n,a1_n,a2_n,features,label_e
0,2018-03-19T02:49:40.9200000Z,pm1-353,20.0,101.0,1000.0,93.984369,1999.998863,200.0,67.0,83.0,...,0.093984,1.999999,0.2,0.06700,0.08300,2.130736,1.234923,1.104255,"[2130.73611212, 2.13073611212, 1234.92292531, ...",3
1,2018-03-19T02:49:40.9820000Z,pm1-356,20.0,101.0,800.0,74.463702,1199.999534,160.0,53.0,27.0,...,0.093080,1.499999,0.2,0.06625,0.03375,2.660304,1.501537,1.342148,"[2128.24325483, 2.66030406854, 1201.22920143, ...",6
2,2018-03-19T02:49:41.9410000Z,pm1-353,20.0,101.0,1000.0,93.985845,1999.998864,200.0,67.0,33.0,...,0.093986,1.999999,0.2,0.06700,0.03300,2.128995,1.209310,1.103185,"[2128.99464918, 2.12899464918, 1209.30977722, ...",3
3,2018-03-19T02:49:41.9880000Z,pm1-356,20.0,101.0,800.0,74.465005,1199.999534,160.0,53.0,67.0,...,0.093081,1.499999,0.2,0.06625,0.08375,2.663272,1.539738,1.375497,"[2130.61728994, 2.66327161242, 1231.79044275, ...",6
4,2018-03-19T02:49:42.9280000Z,pm1-356,20.0,101.0,800.0,74.466307,1199.999535,160.0,53.0,67.0,...,0.093083,1.499999,0.2,0.06625,0.08375,2.663159,1.539378,1.375902,"[2130.52747399, 2.66315934249, 1231.50251145, ...",6


In [8]:
model_type = 'RandomForest' # Use 'DecisionTree', or 'RandomForest'

# train a model.
if model_type == 'DecisionTree':
    model = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",
                                      # Maximum depth of the tree. (>= 0) 
                                      # E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'
                                      maxDepth=15,
                                      # Max number of bins for discretizing continuous features. 
                                      # Must be >=2 and >= number of categories for any categorical feature.
                                      maxBins=32, 
                                      # Minimum number of instances each child must have after split. 
                                      # If a split causes the left or right child to have fewer than 
                                      # minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.
                                      minInstancesPerNode=1, 
                                      # Minimum information gain for a split to be considered at a tree node.
                                      minInfoGain=0.0, 
                                      # Criterion used for information gain calculation (case-insensitive). 
                                      # Supported options: entropy, gini')
                                      impurity="gini")

    ##=======================================================================================================================
    #elif model_type == 'GBTClassifier':
    #    cls_mthd = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
    ##=======================================================================================================================
else:    
    model = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", 
                                      # Passed to DecisionTreeClassifier
                                      maxDepth=15, 
                                      maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0,
                                      impurity="gini",
                                      # Number of trees to train (>= 1)
                                      numTrees=50, 
                                      # The number of features to consider for splits at each tree node. 
                                      # Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].
                                      featureSubsetStrategy="sqrt", 
                                      # Fraction of the training data used for learning each 
                                      # decision tree, in range (0, 1].' 
                                      subsamplingRate = 0.632)

# chain indexers and model in a Pipeline
pipeline_cls_mthd = Pipeline(stages=[labelIndexer, featureIndexer, model])

# train model.  This also runs the indexers.
model_pipeline = pipeline_cls_mthd.fit(training)

In [9]:
# make predictions. The Pipeline does all the same operations on the test data
predictions = model_pipeline.transform(test)

# Create the confusion matrix for the multiclass prediction results
# This result assumes a decision boundary of p = 0.5
conf_table = predictions.stat.crosstab('indexedLabel', 'prediction')
confuse = conf_table.toPandas().sort_values(by=['indexedLabel_prediction'])
confuse.head()

,indexedLabel_prediction,0.0,1.0,2.0,3.0
3,0.0,3506,0,0,0
1,1.0,0,3489,0,0
0,2.0,0,1,3550,0
2,3.0,0,0,0,3625


In [10]:
model_pipeline.write().overwrite().save('../model')

import tarfile

tar = tarfile.open("../model.tar.gz", "w:gz")
tar.add("../model", arcname="model")
tar.close()